In [1]:
import os
%pwd

'/home/towet/Desktop/OpenProjects/MRI-CT-Translator/research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'/home/towet/Desktop/OpenProjects/MRI-CT-Translator'

In [4]:
from pathlib import Path
from dataclasses import dataclass

# Define the evaluation configuration
@dataclass
class EvaluationConfig:
    model_path_AtoB: Path
    model_path_BtoA: Path
    test_dataset_path: Path
    mlflow_uri: str
    all_params: dict

In [5]:

from MRICTTranslator.constants import *
from MRICTTranslator.utils.common import read_yaml, create_directories, save_json

2024-06-07 12:16:21.007340: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-07 12:16:22.887569: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-07 12:16:43.841346: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        
        dataset = 'artifacts/data_ingestion/dataset/Dataset/images/'
        eval_config = EvaluationConfig(
            model_path_AtoB="artifacts/training/g_model_AtoB_001742.h5",
            model_path_BtoA='artifacts/training/g_model_BtoA_001742.h5',
            mlflow_uri="https://dagshub.com/Towet-Tum/MRI-CT-Translator.mlflow",
            all_params=self.params,
            test_dataset_path=str(dataset)
        )
        return eval_config

In [8]:
import os
from os import listdir
from numpy import asarray
import tensorflow as tf
import mlflow
import mlflow.keras
from dagshub import dagshub_logger
from pathlib import Path
from dataclasses import dataclass
from urllib.parse import urlparse
import json
from MRICTTranslator.utils.common import preprocess_data, load_images
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from MRICTTranslator.utils.common import InstanceNormalization

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    @staticmethod
    def load_model(path: str, cust=None) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
 
    
    def load_and_process_dataset(self):
        # dataA is the CT scans and dataB is the MRI scans
        dataA = load_images(self.config.test_dataset_path + 'testA/')
        dataB = load_images(self.config.test_dataset_path + 'testB/')
        # load image data
        data = [dataA, dataB]
        dataset = preprocess_data(data)
        
        return dataset
    
    def evaluation(self, model_path, direction):
        cust = {'InstanceNormalization': InstanceNormalization}
        test_ds = self.load_and_process_dataset()
        self.model = self.load_model(model_path, cust)
        self.score = self.model.evaluate(test_ds)
        self.save_score(direction)
        
    def save_score(self, direction):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path(f"scores_{direction}.json"), data=scores)
        
    
    def log_into_mlflow(self, direction):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with dagshub_logger() as logger, mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            with open(f"scores_{direction}.json", "r") as f:
                scores = json.load(f)
                mlflow.log_metrics(scores)
                logger.log_metrics(scores)
            model_path = self.config.model_path_AtoB if direction == "AtoB" else self.config.model_path_BtoA
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name=f"CycleGAN-{direction}")
            else:
                mlflow.keras.log_model(self.model, "model")
    

In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation(eval_config.model_path_AtoB, "AtoB")
    evaluation.log_into_mlflow("AtoB")
    evaluation.evaluation(eval_config.model_path_BtoA, "BtoA")
    evaluation.log_into_mlflow("BtoA")

except Exception as e:
    raise e

[2024-06-07 12:23:25,575: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-07 12:23:25,691: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-07 12:23:25,693: INFO: common: created directory at: artifacts]


ValueError: Unknown layer: 'InstanceNormalization'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.